In [2]:
%matplotlib inline

# 자동미분

``autograd`` 패키지 : Tensor의 모든 연산에 대해 자동 미분 제공  
실행 - 기반 - 정의(define-by-run) 프레임워크  
코드를 어떻게 작성하여 실행하느냐에 따라 역전파가 정의된다는 뜻  


##Tensor
패키지 중심에는 ``torch.Tensor``클래스 존재  
``.requires_grad`` 속성을 ``True``로 설정 : tensor에서 이뤄진 모든 연산을 추적  
계산 완료 후 ``backward()``호출 : 모든 변화도(gradient) 자동 계산  
이 tensor 변화도 ``.grad``속성에 누적

Tensor 기록 추적을 중단하게 하려면 ``.detach()``호출, 연산 기록으로부터 분리  
``with tocrch.no_grad():``기록 추적(+메모리 사용)을 방지를 위해 코드 블럭 감싸기  
== 변화도는 필요 없지만 requires_grad=True 설정되어 매개변수를 갖는 모델을 평가할 때 유용  


### Funchtion 클래스
Function과 Tensor는 서로 연결되어 있어 모든 연산과정을 부호화하여 순환하지 않는 그래프(acyclic graph)를 생성  
각 tensor는 .grad_fn 속성을 가지는데, 이는 Tensor를 생성한 Function을 참조함
(사용자가 만든 Tensor는 예외로 grad_fn은 None)  

도함수를 계산하기 위해서는 Tensor 의 .backward() 를 호출  
만약 Tensor가 스칼라(하나의 요소 값만 가짐)인 경우 backward에 인자를 정해줄 필요 없음.  
여러 개의 요소를 가지고 있는 경우 tensor 모양을 gradient의 인자로 지정해야 함

In [3]:
import torch

tensor 생성  
``set requires_grad=True``설정 == 연산기록



In [4]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


텐서에 연산 수행



In [5]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


y는 연산의 결과로 생성된 것이므로 ``grad_fn``를 가짐



In [6]:
print(y.grad_fn)

y에 다른 연산 수행



In [7]:
z = y * y * 3
out = z.mean()

print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


``.requires_grad_( ... )``는 기존 Tensor의 ``requires_grad``를 바꿔치기(in-place) 하여 변경  
입력 값이 지정되지 않으면 기본값은 False

In [11]:
a = torch.randn(2, 2) # torch 생성
print(a)
a = ((a * 3) / (a - 1)) # 값 넣어줌
print(a.requires_grad) # 입력값이 지정되지 않으므로 False화
a.requires_grad_(True) # 바꿔치기 해서 넣음
print(a.requires_grad) # True로 출력됨
b = (a * a).sum()
print(b.grad_fn)

tensor([[ 0.9028, -1.1968],
        [-0.9403, -0.9543]])
False
True


## 변화도 Gradients

역전파 할 예정  
out은 스칼라 값만 가지고 있어서  
``out.backward()``은 ``out.backward(torch.tensor(1))``와 동일



In [9]:
out.backward()

변화도 d(out)/dx 출력




In [10]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


4.5값으로 이뤄진 행렬 확인  
Tensor를 o라고 할때  
$o = \frac{1}{4}\sum_i z_i$  
$z_i = 3(x_i+2)^2$  
따라서 $z_i\bigr\rvert_{x_i=1} = 27$.

$\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i+2)$  
$\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$.



야코비안 행렬(Jacobian Matrix): 벡터 함수 y⃗ =f(x⃗ ) 에서 x⃗  에 대한 y⃗ 의 변화도  
벡터-야코비안 곱의 이러한 특성은 스칼라가 아닌 출력을 갖는 모델에 외부 변화도를 제공(feed)하는 것을 매우 편리하게 해줌  

벡터-야코비안 곱의 예제

In [15]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

tensor([ 759.5015,  240.4281, -771.1067], grad_fn=<MulBackward0>)


이제 y는 더이상 스칼라 값이 아님.  
torch.autograd는 전체 야코비안을 직접계산할 수 없지만  
벡터-야코비안 곲은 간단히 backward에 해당 벡터를 인자로 제공하여 얻을 수 있음

In [16]:
gradients = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(gradients)

print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


``with torch.no_grad():``으로 코드를 감싸 autograd가 ``.requires_grad=True``인 Tensor 연산 기록 추적을 멈출 수 있게 함



In [17]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
	print((x ** 2).requires_grad)

True
True
False
